# F-16 Simulation Class

In [ ]:
from pierpont import ppont
from pierpont import daveML

###############################################################################
class OblateEom(ppont.Integrator):
    """
    """
    timeStep = 0.1
    
    # Earth rotatation in body frame
    Per = 0
    Qer = 0
    Rer = 0
    
    Integrator = ppont.Integrator()
    
    # state values: quaternion, position, acceleration and angular rates
    X = []
    
    # the state differential equations
    Xdot = []
    
    # the indices of the state list
    Qni = 0
    Qxi = 1
    Qyi = 2
    Qzi = 3

    Xi = 4
    Yi = 5
    Zi = 6

    Vxi = 7
    Vyi = 8
    Vzi = 9

    Pi = 10
    Qi = 11
    Ri = 12
        
    def Qstate(self,state):
        q0 = state[self.Qni]
        q1 = state[self.Qxi]
        q2 = state[self.Qyi]
        q3 = state[self.Qzi]
        p = state[self.Pi] - self.Per
        q = state[self.Qi] - self.Qer
        r = state[self.Ri] - self.Rer
        return q0, q1, q2, q3, p, q, r
    def QnDot(self, state):
        q0, q1, q2, q3, p, q, r = self.Qstate(state)
        qnDot = -0.5*(q1*p + q2*q + q3*r)
        return qnDot
    def QxDot(self, state):
        q0, q1, q2, q3, p, q, r = self.Qstate(state)
        qxDot = 0.5*(q0*p + q2*r - q3*q)
        return qxDot
    def QyDot(self, state):
        q0, q1, q2, q3, p, q, r = self.Qstate(state)
        qyDot = 0.5*(q0*q - q1*r + q3*p )
        return qyDot
    def QzDot(self, state):
        q0, q1, q2, q3, p, q, r = self.Qstate(state)
        qzDot = 0.5*(q0*r + q1*q - q2*p)
        return qzDot
    
    def PxDot(self, state):
        return state[self.Vxi]
    def PyDot(self, state):
        return state[self.Vyi]
    def PzDot(self, state):
        return state[self.Vzi]
    
    def VxDot(self, state):
        w = self.Planet.RotationRate
        mass = self.CheckMass("VxDot mass is 0")
        ax = self.QforceEcf.X / mass
        xDot = ax + 2.0 * w * state[self.Vyi] + self.Gx + state[self.Xi] * w**2
        return xDot
    def VyDot(self, state):
        w = self.Planet.RotationRate
        mass = self.CheckMass("VyDot mass is 0")
        ay = self.QforceEcf.Y / mass
        yDot = ay - 2.0 * w * state[self.Vxi] + self.Gy + state[self.Yi] * w**2 
        return yDot
    def VzDot(self, state):
        mass = self.CheckMass("VzDot mass is 0")
        az = self.QforceEcf.Z / mass
        return (az + self.Gz)
    
    def Wstate(self, state):
        P = state[self.Pi]
        Q = state[self.Qi]
        R = state[self.Ri]
        Jx = self.gvJx
        Jy = self.gvJy
        Jz = self.gvJz
        Jxz = self.gvJxz
        Gamma = self.Gamma
        l = self.aeroBodyMoment.X
        m = self.aeroBodyMoment.Y
        n = self.aeroBodyMoment.Z
        return P, Q, R, Jx, Jy, Jz, Jxz, Gamma, l, m, n
    def Pdot(self, state):
        P, Q, R, Jx, Jy, Jz, Jxz, Gamma, l, m, n = self.Wstate(state)
        assert Gamma != 0, "Pdot Gamma is 0"
        pDot = (Jxz * (Jx - Jy + Jz)*P*Q - (Jz*(Jz - Jy) + Jxz*Jxz)*Q*R + Jz*l + Jxz*n) / Gamma
        return pDot
    def Qdot(self, state):
        P, Q, R, Jx, Jy, Jz, Jxz, Gamma, l, m, n = self.Wstate(state)
        assert Jy != 0.0, "Qdot Jy is 0"
        qDot = ((Jz - Jx)*P*R - Jxz*(P*P - R*R) + m) / Jy
        return qDot
    def Rdot(self, state):
        P, Q, R, Jx, Jy, Jz, Jxz, Gamma, l, m, n = self.Wstate(state)
        assert Gamma != 0.0, "Rdot Gamma is 0"
        rDot = (((Jx - Jy)*Jx + Jxz*Jxz)*P*Q - Jxz*(Jx - Jy + Jz)*Q*R + Jxz*l + Jx*n) / Gamma
        return rDot
    
    def Init(self):
        # state values: quaternion, position, acceleration and angular rates
        self.X = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

        # the state differential equations
        self.Xdot.clear()
        self.Xdot = [self.QnDot, self.QxDot, self.QyDot, self.QzDot,
                     self.PxDot, self.PyDot, self.PzDot,
                     self.VxDot, self.VyDot, self.VzDot,
                     self.Pdot, self.Qdot, self.Rdot]
        
    def Integrate(self):
        # integrate the equations
        self.X = self.Integrator.RungeKutta4(self.timeStep, self.Xdot, self.X)
        

In [1]:
###############################################################################
class F16Sim(ppont.Simulation):
    """
    The software implements the oblate rotating planet EOM as derived in 
    [Stevens and Lewis]
    (https://bcs.wiley.com/he-bcs/Books?action=index&itemId=0471371459&itemTypeId=BKS&bcsId=2073). 
    
    Equations in LaTeX format:
    
    $\dot{q_0} = -0.5 * (Pq_1 + Qq_2 + Rq_3)$  
    $\dot{q_1} = 0.5 * (Pq_0 + Rq_2 - Qq_3)$  
    $\dot{q_2} = 0.5 * (Qq_0 - Rq_1 + Pq_3)$  
    $\dot{q_3} = 0.5 * (Rq_0 + Qq_1 - Pq_2)$  

    $\dot{P_x} = V_x$  
    $\dot{P_y} = V_y$  
    $\dot{P_z} = V_z$  
    where $P$ and $V$ are in the ECEF frame.

    $\dot{v_x} = \frac{F_x}{m} + 2 \omega_e V_y + g_x + P_x \omega^2_e$  
    $\dot{v_y} = \frac{F_y}{m} - 2 \omega_e V_x + g_y + P_y \omega^2_e$  
    $\dot{v_z} = \frac{F_z}{m} + g_z$  
    where $\omega_e$ is the rotation rate of Earth.  The terms $g_x$, $g_y$, 
    and $g_z$ are the $J_2$ gravity components in ECEF. This acceleration equation 
    is in the ECEF frame.

    $\Gamma \dot{P} = J_{xz} [J_x - J_y + J_z]PQ - [J_z(J_z - J_y) + J^2_{xz}]QR 
      + l J_z + n J_{xz}$  
    $J_y \dot{Q} = (J_z - J_x)PR - J_{xz}(P^2 - R^2) + m$  
    $\Gamma \dot{R} = [(J_x - J_y)J_x + J^2_{xz}]PQ - J_{xz}[J_x - J_y + J_z]QR 
      + l J_{xz} n J_x$  
    where $\Gamma = J_x J_z - J^2_{xz}$ 
    """
    EngineModelFile = None
    EngineModel = daveML.Model()
    
    Planet = ppont.Earth()
    RotationAngle = 0
    EarthRotation = ppont.Quaternion(0, 0, 0, Planet.RotationRate)
        
    # Earth rotatation in body frame
    Per = 0
    Qer = 0
    Rer = 0
    
    # quaternion frame rotations
    #  i = inertial frame ECI
    #  e = earth centered, earth fixed ECEF
    #  n = north east down NED
    #  b = body forward right down FRD
    Qe2n = ppont.Quaternion(1,0,0,0)
    Qn2b = ppont.Quaternion(1,0,0,0)
    Qe2b = ppont.Quaternion(1,0,0,0)
    Qi2e = ppont.Quaternion(1,0,0,0)
    
    QforceEcf = ppont.Quaternion(0,0,0,0)
    
    # ECEF gravity components
    Gx = 0
    Gy = 0
    Gz = 0
    
    Integrator = ppont.Integrator()
    
    # state values: quaternion, position, acceleration and angular rates
    X = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
    
    # the state differential equations
    Xdot = []
    
    # the indices of the state list
    Qni = 0
    Qxi = 1
    Qyi = 2
    Qzi = 3
    
    Xi = 4
    Yi = 5
    Zi = 6
    
    Vxi = 7
    Vyi = 8
    Vzi = 9
    
    Pi = 10
    Qi = 11
    Ri = 12
    
    def SetEngineModelFile(self, engineModelFile):
        self.EngineModelFile = engineModelFile
        
    def EvaluateAeroModel(self):
        for d in self.EngineModelInput:
            self.EngineModel.Set(d, self.Data[d])
        self.EngineModel.Update()
    
    def Qstate(self,state):
        q0 = state[self.Qni]
        q1 = state[self.Qxi]
        q2 = state[self.Qyi]
        q3 = state[self.Qzi]
        p = state[self.Pi] - self.Per
        q = state[self.Qi] - self.Qer
        r = state[self.Ri] - self.Rer
        return q0, q1, q2, q3, p, q, r
    def QnDot(self, state):
        q0, q1, q2, q3, p, q, r = self.Qstate(state)
        qnDot = -0.5*(q1*p + q2*q + q3*r)
        return qnDot
    def QxDot(self, state):
        q0, q1, q2, q3, p, q, r = self.Qstate(state)
        qxDot = 0.5*(q0*p + q2*r - q3*q)
        return qxDot
    def QyDot(self, state):
        q0, q1, q2, q3, p, q, r = self.Qstate(state)
        qyDot = 0.5*(q0*q - q1*r + q3*p )
        return qyDot
    def QzDot(self, state):
        q0, q1, q2, q3, p, q, r = self.Qstate(state)
        qzDot = 0.5*(q0*r + q1*q - q2*p)
        return qzDot
    
    def PxDot(self, state):
        return state[self.Vxi]
    def PyDot(self, state):
        return state[self.Vyi]
    def PzDot(self, state):
        return state[self.Vzi]
    
    def VxDot(self, state):
        w = self.Planet.RotationRate
        mass = self.CheckMass("VxDot mass is 0")
        ax = self.QforceEcf.X / mass
        xDot = ax + 2.0 * w * state[self.Vyi] + self.Gx + state[self.Xi] * w**2
        return xDot
    def VyDot(self, state):
        w = self.Planet.RotationRate
        mass = self.CheckMass("VyDot mass is 0")
        ay = self.QforceEcf.Y / mass
        yDot = ay - 2.0 * w * state[self.Vxi] + self.Gy + state[self.Yi] * w**2 
        return yDot
    def VzDot(self, state):
        mass = self.CheckMass("VzDot mass is 0")
        az = self.QforceEcf.Z / mass
        return (az + self.Gz)
    
    def Wstate(self, state):
        P = state[self.Pi]
        Q = state[self.Qi]
        R = state[self.Ri]
        Jx = self.gvJx
        Jy = self.gvJy
        Jz = self.gvJz
        Jxz = self.gvJxz
        Gamma = self.Gamma
        l = self.aeroBodyMoment.X
        m = self.aeroBodyMoment.Y
        n = self.aeroBodyMoment.Z
        return P, Q, R, Jx, Jy, Jz, Jxz, Gamma, l, m, n
    def Pdot(self, state):
        P, Q, R, Jx, Jy, Jz, Jxz, Gamma, l, m, n = self.Wstate(state)
        assert Gamma != 0, "Pdot Gamma is 0"
        pDot = (Jxz * (Jx - Jy + Jz)*P*Q - (Jz*(Jz - Jy) + Jxz*Jxz)*Q*R + Jz*l + Jxz*n) / Gamma
        return pDot
    def Qdot(self, state):
        P, Q, R, Jx, Jy, Jz, Jxz, Gamma, l, m, n = self.Wstate(state)
        assert Jy != 0.0, "Qdot Jy is 0"
        qDot = ((Jz - Jx)*P*R - Jxz*(P*P - R*R) + m) / Jy
        return qDot
    def Rdot(self, state):
        P, Q, R, Jx, Jy, Jz, Jxz, Gamma, l, m, n = self.Wstate(state)
        assert Gamma != 0.0, "Rdot Gamma is 0"
        rDot = (((Jx - Jy)*Jx + Jxz*Jxz)*P*Q - Jxz*(Jx - Jy + Jz)*Q*R + Jxz*l + Jx*n) / Gamma
        return rDot
    
    def Trim(self):
        return
    
    def Reset(self, ic):
        self.ResetSimulation(ic)
        
        if self.EngineModelFile is not None:
            self.EngineModel.LoadDml(self.EngineModelFile, False)
        
        self.RotationAngle = 0
        
        self.Planet.Latitude = self.SetValue("latitude")
        self.Planet.Longitude = self.SetValue("longitude")
        self.Planet.Altitude = self.SetValue("altitudeMsl")
        [x, y, z] = self.Planet.LlaToPcpf()
        self.Position.X = x
        self.Position.Y = y
        self.Position.Z = z
        
        # initialize the frd/ecf quaternion
        roll  = self.BodyAngle.X
        pitch = self.BodyAngle.Y
        yaw   = self.BodyAngle.Z
        lat = self.Planet.Latitude
        lon = self.Planet.Longitude
        self.Qe2b.SetQfrdWrtEcf(roll , pitch , yaw, lat, lon)
        
        # transform u,v,w to ECEF velocities
        Vecf = ppont.Vector3(0,0,0)
        Vecf = self.Qe2b * self.BodyVelocity * ~self.Qe2b
        
        self.Xdot.clear()
        self.Xdot = [self.QnDot, self.QxDot, self.QyDot, self.QzDot,
                     self.PxDot, self.PyDot, self.PzDot,
                     self.VxDot, self.VyDot, self.VzDot,
                     self.Pdot, self.Qdot, self.Rdot] 
        
        self.X[self.Qni] = self.Qe2b.N
        self.X[self.Qxi] = self.Qe2b.X
        self.X[self.Qyi] = self.Qe2b.Y
        self.X[self.Qzi] = self.Qe2b.Z
        
        self.X[self.Xi] = self.Position.X
        self.X[self.Yi] = self.Position.Y
        self.X[self.Zi] = self.Position.Z
        
        self.X[self.Vxi] = Vecf.X
        self.X[self.Vyi] = Vecf.Y
        self.X[self.Vzi] = Vecf.Z
        print("Vecf: ", Vecf.X, Vecf.Y, Vecf.Z)
        
        self.X[self.Pi] = self.BodyAngularRate.X
        self.X[self.Qi] = self.BodyAngularRate.Y
        self.X[self.Ri] = self.BodyAngularRate.Z
        
    def Operate(self):
        # create quaternions
 
        # TODO: need a check case the Q rotations
        # set q frd/ecf (e2b) ECF to body
        self.Qe2b.N = self.X[self.Qni]
        self.Qe2b.X = self.X[self.Qxi]
        self.Qe2b.Y = self.X[self.Qyi]
        self.Qe2b.Z = self.X[self.Qzi]
        
        # set q ned/ecf (e2n) ECF to NED
        self.Qe2n.SetLatLon(self.Planet.Latitude, self.Planet.Longitude)
        
        # set q frd/ned (n2b) NED to body
        self.Qn2b = ~self.Qe2n * self.Qe2b
        
        # get the euler angles from the quaternion
        [roll, pitch, yaw] = self.Qn2b.EulerAnglesFromQ()
        
        # rotate the ECF position to ECI to get the inertial position
        self.Qi2e.SetPlanetRotation(self.RotationAngle)
        qgePosition = ppont.Quaternion( 0, self.X[self.Xi], self.X[self.Yi], self.X[self.Zi] )
        qeiPosition = self.Qi2e * qgePosition * ~self.Qi2e
        
        # save output data
        self.RecData.Add('altitudeMsl_m', self.Planet.Altitude)
        self.RecData.Add('latitude_rad', self.Planet.Latitude)
        self.RecData.Add('longitude_rad', self.Planet.Longitude)
        self.RecData.Add('gePosition_m_X', self.X[self.Xi])
        self.RecData.Add('gePosition_m_Y', self.X[self.Yi])
        self.RecData.Add('gePosition_m_Z', self.X[self.Zi])
        self.RecData.Add('eulerAngle_rad_Roll', roll)
        self.RecData.Add('eulerAngle_rad_Pitch', pitch)
        self.RecData.Add('eulerAngle_rad_Yaw', yaw)
        self.RecData.Add('trueAirspeed_m_s', self.trueAirspeed)
        self.RecData.Add('eiPosition_m_X', qeiPosition.X)
        self.RecData.Add('eiPosition_m_Y', qeiPosition.Y)
        self.RecData.Add('eiPosition_m_Z', qeiPosition.Z)
        
        # get earth rotation in the body frame
        wEarthFrd = ~self.Qe2b * self.EarthRotation * self.Qe2b
        
        # set the Earth rotation in the body frame
        self.Per = wEarthFrd.X
        self.Qer = wEarthFrd.Y
        self.Rer = wEarthFrd.Z
        
        x = self.X[self.Xi]
        y = self.X[self.Yi]
        z = self.X[self.Zi]
        [self.Gx, self.Gy, self.Gz] = self.Planet.GravityJ2( x, y, z )
        g = ppont.Vector3(self.Gx, self.Gy, self.Gz)
        self.RecData.Add('localGravity_m_s2', g.Magnitude())

        # integrate the equations
        self.X = self.Integrator.RungeKutta4(self.timeStep, self.Xdot, self.X)
        
        # advance time and set up for next integration
        self.AdvanceTime()
        
        # Compute the aerodynamic loads from the DAVE-ML model
        #  set the DAVE-ML model inputs
        vel = ppont.Vector3(self.X[self.Vxi], self.X[self.Vyi], self.X[self.Vzi])
        self.trueAirspeed = vel.Magnitude()
        assert self.trueAirspeed != 0, "TrueAirspeed is 0 to model"
        # calculate alpha and beta
        uvw = ~self.Qe2b * vel * self.Qe2b
        self.Data["angleOfAttack"] = math.atan2(uvw.Z, uvw.X)
        self.Data["angleOfSideslip"] = math.atan2(uvw.Y, math.sqrt(uvw.X**2 + uvw.Z**2))
        self.Data["trueAirspeed"] = self.trueAirspeed * self.MeterToFeet
        self.Data["bodyAngularRate_Roll"] = self.X[self.Pi]
        self.Data["bodyAngularRate_Pitch"] = self.X[self.Qi]
        self.Data["bodyAngularRate_Yaw"] = self.X[self.Ri]
        self.EvaluateAeroModel()
        
        # get dynamic pressure:  q = 1/2 rho v^2
        dynamicPressure = (
            self.Planet.DynamicPressure(self.Planet.Altitude, self.trueAirspeed)
        )
        self.RecData.Add('speedOfSound_m_s', self.Planet.speedOfSoundMps)

        # Get the qS factor for getting dimensional forces and moments
        qS = dynamicPressure * self.referenceWingArea
        
        # compute the aero forces in the body frame
        self.CalcAeroBodyForces(qS)
        
        # rotate body forces to the ECEF frame
        self.QforceEcf = self.Qe2b * self.aeroBodyForce * ~self.Qe2b
        
        # calculate the dimensional aero moments
        self.CalcAeroBodyMoments(qS)
        
        # update the latitude, longitude and altitude from ECEF X, Y, Z position
        self.Planet.PcpfToLlaZhu(self.X[self.Xi], self.X[self.Yi], self.X[self.Zi])
        
        # rotate the earth
        self.RotationAngle = self.Planet.RotationRate * self.time

# Sphere check

In [2]:
%%time
from pierpont import ppont
import math
#
ic = {
    "altitudeMsl": [30000, "ft"]
}
ball = F16Sim('models/sphere_dragless.dml')
ball.Reset(ic)
ball.Run(30.0)

*******************************************
Model:  Dragless Sphere
creation date:  2022-12-28
file version:  Initial version
*******************************************

+++++ MODEL INPUTS AND OUTPUTS +++++
++++++++++++++++++++++++++++++++++++

----- DAVE-ML MODEL PARSE COMPLETE -----
========= SetIC ==============
{'altitudeMsl': 9144.0}
++ timeStep = 0.1 [default]
++ totalMass = 14.593902937 [DML model]
++ referenceWingSpan = 0 [default]
++ referenceWingChord = 0 [default]
++ referenceWingArea = 0 [default]
++ trueAirspeed = 0 [default]
++ angleOfAttack = 0 [default]
++ angleOfSideslip = 0 [default]
Vuvw:  (0.0, 0.0, 0.0)
++ eulerAngle_Roll = 0 [default]
++ eulerAngle_Pitch = 0 [default]
++ eulerAngle_Yaw = 0 [default]
++ eulerAngleRate_Roll = 0 [default]
++ eulerAngleRate_Pitch = 0 [default]
++ eulerAngleRate_Yaw = 0 [default]
++ bodyMomentOfInertia_X = 4.88094466281336 [DML model]
++ bodyProductOfInertia_XY = 0 [default]
++ bodyProductOfInertia_XZ = 0 [default]
++ bodyProductOfIne

In [3]:
from pierpont import util
#
gvImperialData = \
{
    'altitudeMsl_ft': ['ft', 'm'], 
    'longitude_deg': ['deg', 'rad'],
    'latitude_deg': ['deg', 'rad'],
    'localGravity_ft_s2': ['ft_s2', 'm_s2'],
    'gePosition_ft_X': ['ft', 'm'], 
    'gePosition_ft_Y': ['ft', 'm'], 
    'gePosition_ft_Z': ['ft', 'm'],
    'eulerAngle_deg_Yaw': ['deg', 'rad'],
    'eulerAngle_deg_Pitch': ['deg', 'rad'],
    'eulerAngle_deg_Roll': ['deg', 'rad'],
    'speedOfSound_ft_s': ['ft_s', 'm_s']
}
ball.RecData.CreateImperialData(gvImperialData)
#
checkFile = (
    "NESC-check-cases/Atmospheric_checkcases/Atmos_01_DroppedSphere/"
    "Atmos_01_sim_01.csv"
)
gvCC1 = util.GetNESCData(checkFile)
#
util.PrintErrorTable("Dragless Sphere: Oblate Earth", gvImperialData, 
                     ball, gvCC1)

number of headers:  31
['time', 'gePosition_ft_X', 'gePosition_ft_Y', 'gePosition_ft_Z', 'feVelocity_ft_s_X', 'feVelocity_ft_s_Y', 'feVelocity_ft_s_Z', 'altitudeMsl_ft', 'longitude_deg', 'latitude_deg', 'localGravity_ft_s2', 'eulerAngle_deg_Yaw', 'eulerAngle_deg_Pitch', 'eulerAngle_deg_Roll', 'bodyAngularRateWrtEi_deg_s_Roll', 'bodyAngularRateWrtEi_deg_s_Pitch', 'bodyAngularRateWrtEi_deg_s_Yaw', 'altitudeRateWrtMsl_ft_min', 'speedOfSound_ft_s', 'airDensity_slug_ft3', 'ambientPressure_lbf_ft2', 'ambientTemperature_dgR', 'aero_bodyForce_lbf_X', 'aero_bodyForce_lbf_Y', 'aero_bodyForce_lbf_Z', 'aero_bodyMoment_ftlbf_L', 'aero_bodyMoment_ftlbf_M', 'aero_bodyMoment_ftlbf_N', 'mach', 'dynamicPressure_lbf_ft2', 'trueAirspeed_nmi_h']
Dragless Sphere: Oblate Earth
Variable                  L2      L-Inf   Frechet
--------                  --      -----   --------
altitudeMsl_ft            0.133   0.02    NC     
longitude_deg             0.0     0.0     NC     
latitude_deg              0.0     

# F-16 sim

In [4]:
%%time
from pierpont import ppont
#
# 36.01916667, -75.67444444, 10013
ic = {
    "trueAirspeed": [335.15, "kt"],
    "angleOfAttack": [0, "deg"],
    "eulerAngle_Yaw": [45, "deg"],
    "latitude": [36.01916667, "deg"],
    "longitude": [-75.67444444, "deg"],
    "altitudeMsl": [10013, "ft"]
}
#
aeroInputs = ["trueAirspeed", 
              "angleOfAttack", "sideSlip",
              "bodyAngularRate_Roll", "bodyAngularRate_Pitch", "bodyAngularRate_Yaw"
              "el", "ail", "rdr", "xcg"]
#
engineInputs = [ 'PWR', 'altitudeMsl', 'mach' ]
#
f16sim = F16Sim('models/F16/F16_aero.dml')
f16sim.SetEngineModelFile('models/F16/F16_prop.dml')
f16sim.Reset(ic)
f16sim.AddAeroModelInput(aeroInputs)
f16sim.AddEngineModelInput(engineInputs)
#f16sim.Run(180)

*******************************************
Model:  F-16 Subsonic Aerodynamics Model (a la Garza)
creation date:  2003-06-10
file version:  $ Revision: 395 $
*******************************************

+++++ MODEL INPUTS AND OUTPUTS +++++
++> Input:  vt
++> Input:  alpha
++> Input:  beta
++> Input:  p
++> Input:  q
++> Input:  r
++> Input:  el
++> Input:  ail
++> Input:  rdr
++> Input:  xcg
++> Output:  cx
++> Output:  cy
++> Output:  cz
++> Output:  cl
++> Output:  cm
++> Output:  cn
++++++++++++++++++++++++++++++++++++

----- DAVE-ML MODEL PARSE COMPLETE -----
========= SetIC ==============
{'trueAirspeed': 172.41604066, 'angleOfAttack': 0.0, 'eulerAngle_Yaw': 0.7853981633974483, 'latitude': 0.6286530522161018, 'longitude': -1.3207682150955165, 'altitudeMsl': 3051.9624000000003}
++ timeStep = 0.1 [default]
++ totalMass = 1 [default]
++ referenceWingSpan = 9.144 [DML model]
++ referenceWingChord = 3.450336 [DML model]
++ referenceWingArea = 0 [default]
++ trueAirspeed = 172.41604066 

## Check

In [5]:
from pierpont import util
#
#f16sim.RecData.CreateImperialData(gvImperialData)
#
checkFile = (
    "NESC-check-cases/Atmospheric_checkcases/Atmos_11_TrimCheckSubsonicF16/"
    "Atmos_11_sim_02.csv"
)
gvCC11 = util.GetNESCData(checkFile)
#

number of headers:  27
['time', 'feVelocity_ft_s_X', 'feVelocity_ft_s_Y', 'feVelocity_ft_s_Z', 'altitudeMsl_ft', 'longitude_deg', 'latitude_deg', 'localGravity_ft_s2', 'eulerAngle_deg_Yaw', 'eulerAngle_deg_Pitch', 'eulerAngle_deg_Roll', 'bodyAngularRateWrtEi_deg_s_Roll', 'bodyAngularRateWrtEi_deg_s_Pitch', 'bodyAngularRateWrtEi_deg_s_Yaw', 'altitudeRateWrtMsl_ft_min', 'speedOfSound_ft_s', 'airDensity_slug_ft3', 'ambientPressure_lbf_ft2', 'ambientTemperature_dgR', 'aero_bodyForce_lbf_X', 'aero_bodyForce_lbf_Y', 'aero_bodyForce_lbf_Z', 'aero_bodyMoment_ftlbf_L', 'aero_bodyMoment_ftlbf_M', 'aero_bodyMoment_ftlbf_N', 'mach', 'trueAirspeed_nmi_h']
